<a href="https://colab.research.google.com/github/youkiti/ARE/blob/main/Data_extraction_from_fulltext_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This is a code to simplify the full-text review of a PDF file using OpenAI's GPT-4 API.

##Inspiration: https://onlinelibrary.wiley.com/doi/10.1002/jrsm.1689?af=R
##Requirements:

*   pdf files in Google drive
*   OpenAI API key





In [ ]:
!pip install -q openai pypdf2 llmx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#configuration

In [ ]:
from openai import OpenAI
import json
import pandas as pd
import os
import glob
import numpy as np
import PyPDF2
from tqdm import tqdm
from google.colab import userdata

#OPEN AI key
API_KEY = userdata.get('opanai_API_key')
client = OpenAI(api_key=API_KEY)

#If you use Azure
"""
#key
API_KEY = userdata.get('Azure_API_key')
RESOURCE_ENDPOINT = userdata.get('RESOURCE_ENDPOINT')

#client
client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version="2023-12-01-preview",
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=RESOURCE_ENDPOINT,
    api_key=API_KEY
)

"""


#deploy names
deployment_ids = ["gpt-4-1106-preview","gpt-3.5-turbo-1106","text-embedding-ada-002"]

#data preparation

In [ ]:
folder_path = "YOUR OWN Google Drive PATH, NOT the URL"
os.chdir(folder_path)
pdf_files = glob.glob(os.path.join(folder_path, '*.pdf'))

In [ ]:
print(f"the number of pdf files:{len(pdf_files)}")

In [ ]:
#extract text
def pdf_extract(pdf_file):
  data = []
  with open(pdf_file, 'rb') as file:
      reader = PyPDF2.PdfReader(file)
      # concatenate
      text = ''
      for page in reader.pages:
        text += page.extract_text()
  return text

#Process of GPT

In [ ]:
#To modify the function to extract specific information from a PDF, you should adjust it according to your needs. Ensure that the modifications align with the function calling and settings you plan to use.

system_message = """
You are a systematic reviewer. Please extract the key features from the user inputted research article.
Specifically, user need information about the title of the article, publication year, the names of authors,  the country where the research was conducted, the study design, and the outcomes.
Please extract the concept of the outcomes as defined in the method, and the actual result values separatedly.
Please extract the conclusion of the discussion.
"""




In [ ]:
#Ask GPT to write the JSON of function calling

system_for_function = """
Write a JSON structure for a function specification that includes the name of the function, a description, and detailed parameters based on the requirements listed in the system message. Each parameter should have a type and a brief description. Format the JSON as you would for API documentation.

#example of a system message and JSON structure
system_message = You are a systematic reviewer. Please extract the key features from the user inputted research article. User need information about the title of the article, publication year.

functions=[
    {
        "name":"data_extraction",
        "description":"characteristics of the study",
        "parameters":{
            "type":"object",
            "properties":{
                "title":{
                    "type":"string",
                    "description":"the title of the research article"
                },
                "publication year":{
                    "type":"string",
                    "description":"the publication year of the research article"
                }
            }
        }
    }
]

"""


functions = [
    {
        "name":"json_output",
        "description":" a JSON structure for a function specification",
        "parameters":{
            "type":"object",
            "properties":{
                "json":{
                    "type":"string",
                    "description":"JSON"
                }
            }
        }
    }
]


res = client.chat.completions.create(
    model=deployment_ids[0],
    messages=[
        {"role":"system",
        "content":system_for_function},
        {
        "role": "user",
        "content": system_message
        }],
    functions = functions,
    function_call = "auto",
    temperature = 0
  )

res = res.model_dump_json()
res = json.loads(res)
response = res["choices"][0]["message"]["function_call"]["arguments"]
functions = [json.loads(response)]

#check the outputs
if 'functions' in functions[0]:
  functions = functions[0]['functions']
else:
  functions = functions

print(functions)


↑Even if you use function calling, the output will not be stable, so please visually check that the format is correct.

In [ ]:
def GPT_extract(text):
  res = client.chat.completions.create(
    model=deployment_ids[0],
    messages=[
        {"role":"system",
        "content":system_message},
        {
        "role": "user",
        "content": text
        }],
    functions=functions,
    function_call = "auto",
    temperature = 0
  )
  response_json = res.model_dump_json()
  response_json = json.loads(response_json)
  response_json = response_json["choices"][0]["message"]["function_call"]
  return response_json

In [ ]:
df = pd.DataFrame()
for pdf_file in tqdm(pdf_files):
  text = pdf_extract(pdf_file)
  response_json = GPT_extract(text)
  arguments_str = response_json['arguments']
  arguments_json = json.loads(arguments_str)
  df_pdf = pd.DataFrame([arguments_json])
  df = pd.concat([df, df_pdf])

In [ ]:
#Add the pdf file names
pdf_filenames = [os.path.basename(pdf_path) for pdf_path in pdf_files]
df["pdf_name"] = pdf_filenames

In [ ]:
df.head()

In [ ]:
#save
df.to_csv('data_extraction.csv',index=False, encoding='utf-8')